# Gather Data

Our goal is to bring the data into our notebook with Pandas and DataFrames.  I tend to prefer getting data from a CSV file or similar, but I can also get data from an API which I'll demonstrate.  This generally means:

* Setup common dependencies
* Import CSV
* Import API
* Handle indices and columns
* Sort
* Set data types on columns

## Setup Common Dependencies

I tend to copy around some of the same dependencies from notebook to notebook.  Some of this I don't necessarily use all the time, but the project feels simple and direct.

**Style** One of the options I like to do is set the ggplot style on matplotlib.  It just has a little nicer approach to white space and color.  I also set the figsize, which is a tuple in inches.  I like a larger format so I can show someone in the room what I'm seeing so far.

In [1]:
%matplotlib inline
from IPython.core.pylabtools import figsize
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
plt.style.use('ggplot')
figsize(11,9)

import scipy.stats as stats

import pymc as pm

## Import CSV

The common way to get data into a DataFrame is with `read_csv`.  The seperator can be added on `read_csv` if you have tab delimited or pipe delimited or otherwise delimited data.  There are other useful features like setting index_col for specifying which columns should be used for the index.  The documentation is [found here](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_csv.html).

In [2]:
git_logs_filename = 'data/popular_open_source_logs.csv'
df = pd.read_csv(git_logs_filename)

### Excel Files

Excel file import uses a module called `xlrd`.  Instead of trying to handle that in Pandas, I typically prefer to take a look at these kinds of files in a spreadsheet and then export a CSV.  That's because of the highly variable concept we have about what makes a spreadsheet.  If you do want to use input data as Excel, here is the [documentation](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_excel.html) for that.  There are a lot of tools in there for picking out the table you're after.

In [3]:
alternate_posts_filename = "data/alt_posts-2016-06-06.csv"
alternate_posts = pd.read_csv(alternate_posts_filename)
alternate_posts.head()

,UniversalMessageId,SocialNetwork,SenderUserId,SenderScreenName,SenderListedName,SenderProfileImgUrl,SenderProfileLink,Sender Followers Count,SenderKloutScore,SenderAge,...,Geo Target,Post Id,Associated Cases,Location,Country,State,City,Latitude,Longitude,Sender Email
0,WEB_115_sg_57557f1a111,WEB,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,Germany,NaN,NaN,NaN,0,0,NaN
1,WEB_115_sg_57557ee2a7,WEB,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,Unknown,NaN,NaN,NaN,0,0,NaN
2,WEB_115_sg_57557ddcab,WEB,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,United States,NaN,NaN,NaN,0,0,NaN
3,WEB_115_sg_57557d1174,WEB,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,United States,NaN,NaN,NaN,0,0,NaN
4,WEB_115_sg_57557ca37,WEB,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,United States,NaN,NaN,NaN,0,0,NaN


This data frame is particularly messy and sparse.  We'll have fun with it.

## Import API

Sometimes it's more useful to bring in data from an API and either store it as a CSV or continue to consume the data as I work.  This takes a couple new tools:

* requests
* API token

The [requests library](http://docs.python-requests.org/en/master/) is "HTTP for Humans."  It makes it fairly easy to get data from an API.  
    
`requests` can handle various types of authentication, but we're going to grab some US Census data, so you'll **need an API key for that.**  It takes about 2-3 minutes to get one, and you start [here](http://api.census.gov/data/key_signup.html).

I keep my tokens out of sight and out of the git repository.  That means my `.gitignore` file has a line on it with `.env` on it.  I thien use the [python-dotenv](https://github.com/theskumar/python-dotenv) package to load that data into my scripts.

In [4]:
import requests

from os.path import join, dirname
import os
from dotenv import load_dotenv, find_dotenv

dotenv_path = join(dirname('__file__'), '.env')
load_dotenv(dotenv_path)

API_KEY = os.environ.get("API_KEY")

# This is a County Business Patterns API endpoint
url = "http://api.census.gov/data/2014/cbp?key=%s&get=EMP,ESTAB,EMPSZES,EMPSZES_TTL,PAYANN&for=state:*" % (API_KEY)
result = requests.get(url)
result.reason
cbp = None
if result.ok:
    data = result.json()
    cbp = pd.DataFrame(data[1:], columns=data[0])
print(result.reason)

OK


In [5]:
cbp.head()

,EMP,ESTAB,EMPSZES,EMPSZES_TTL,PAYANN,state
0,1604016,97714,001,All establishments,64291502,01
1,92653,48319,212,Establishments with 1 to 4 employees,3451814,01
2,137019,20741,220,Establishments with 5 to 9 employees,4496433,01
3,185488,13759,230,Establishments with 10 to 19 employees,6423509,01
4,285878,9511,241,Establishments with 20 to 49 employees,10117574,01


## Handle Indices and Columns

Sometimes we can get a more-manageable data frame by figuring out the columns and indices first.

In [6]:
columns = ['timestamp', 'project', 'email', 'lines_inserted', 'lines_removed']
git_logs = pd.read_csv(git_logs_filename, index_col='timestamp', usecols=columns)
git_logs.head()

,project,email,lines_inserted,lines_removed
timestamp,,,,
1034527543,blender,hans@lambermont.dyndns.org,1389,0
1034710347,blender,hans@lambermont.dyndns.org,11,4
1034889370,blender,nlin@nlin.net,1,1
1034951794,blender,nlin@nlin.net,45,21
1034953322,blender,nlin@nlin.net,21829,0


## Sort

The syntax for sort now is `sort_values(by=...)`. We can use this to sort by a non-index value.

In [7]:
git_logs.sort_values(by='project', ascending=False, inplace=True)
git_logs.head()

,project,email,lines_inserted,lines_removed
timestamp,,,,
1465068169,vagrant,sethvargo@gmail.com,1,0
1343442580,vagrant,mitchell.hashimoto@gmail.com,59,56
1342477491,vagrant,mitchell.hashimoto@gmail.com,17,1
1343190758,vagrant,mitchell.hashimoto@gmail.com,1524,1427
1343254261,vagrant,mitchell.hashimoto@gmail.com,1,1


## Set Data Types on Columns

Often I need to help Pandas figure out what's in my data.  You can either set converters as you bring the data in or make transformation on the data once it's arrived.

Also, time columns have special tools like `parse_dates`, `infer_datetime_format`, `keep_date_col`, `date_parser`, and `day_first`.

If you're not sure what you have, you can use dtypes on the data frame to see what you're starting with.

In [8]:
git_logs.dtypes

project           object
email             object
lines_inserted     int64
lines_removed      int64
dtype: object

In [9]:
posts_filename = "data/posts-2016-06-08-21-35-42.csv"
columns = ['Author', 'Time', 'Text', 'ProfileUrl', 'PostUrl', 'Lang',
           'Sentiment']
posts = pd.read_csv(posts_filename, parse_dates=['Time'], index_col='Time', usecols=columns)
posts['HourOfDay'] = posts.index.hour
posts['DayOfWeek'] = posts.index.dayofweek
posts.head()

,Author,Text,ProfileUrl,PostUrl,Lang,Sentiment,HourOfDay,DayOfWeek
Time,,,,,,,,
2016-05-31 02:40:00,Organ Transplant Immunosuppressant Drugs Marke...,Organ Transplant Immunosuppressant Drugs Marke...,http://news.scoopasia.com/index.php/news/organ...,http://news.scoopasia.com/index.php/news/organ...,en,Positive,2,1
2016-05-31 04:00:00,Study show female heart patients less likely t...,Study show female heart patients less likely t...,http://www.eurekalert.org/pub_releases/2016-05...,http://www.eurekalert.org/pub_releases/2016-05...,en,Negative,4,1
2016-05-31 04:30:00,Letter to the editor,"May 31, 2016 \nDeath penalty may be on ballot ...",http://www.starbeacon.com/opinion/letters_to_t...,http://www.starbeacon.com/opinion/letters_to_t...,en,Negative,4,1
2016-05-31 07:10:00,beforeitsnews.com,(Before It's News)\nLow Back Pain Pipeline Mar...,http://beforeitsnews.com/health/2016/05/low-ba...,http://beforeitsnews.com/health/2016/05/low-ba...,en,Positive,7,1
2016-05-31 08:00:00,admin,"By admin | Health , Wellness | 31 May 2016 | A...",https://tsnnews.com/two-smoking-cessation-drug...,https://tsnnews.com/two-smoking-cessation-drug...,en,Positive,8,1


In [10]:
posts.dtypes

Author        object
Text          object
ProfileUrl    object
PostUrl       object
Lang          object
Sentiment     object
HourOfDay      int32
DayOfWeek      int32
dtype: object

**TTD:**

* bring in from a database (use sqlite3)
* merge (chapter 5)
* maybe ODBC driver (??)
* HDF5 files
* Feather format (Apache)
* JSON (format JSON on the request)
* ProtoBuf (??)
* Compressed CSV